In [1]:
%cd ..

/home/radxa/dev/fire-and-smoke-detection


In [2]:
import cv2
import onnxruntime as ort
# import matplotlib.pyplot as plt
# %matplotlib inline

from utils.process import preprocessing, postprocessing

IMAGE_PATH = "data/images/sample_01.png"
ONNX_MODEL_PATH = "models/smoke_tiny_lrelu.onnx"
CLASS_NAMES = ["BG", "Fire", "Smoke"]
COLORS = ["red", "yellow", "white"]
IMAGE_SHAPE = (640, 640)

In [3]:
session = ort.InferenceSession(ONNX_MODEL_PATH)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

In [4]:
image = cv2.imread(IMAGE_PATH)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# preprocessing
tensor_input, image, ratios = preprocessing(image, IMAGE_SHAPE)
print(f"tensor_input shape: {tensor_input.shape}")

# predictions
predictions = session.run([output_name], {input_name: tensor_input})
print(f"predictions shape: {predictions[0].shape}")

print(predictions)

# postprocessing
outputs = postprocessing(predictions[0], ratios)



# plt.figure(figsize=(10, 10))
# plt.imshow(image)

# for detections in outputs:

#     if detections.ndim == 1:
#         detections = detections.reshape(1, -1)

#         for det in detections:
#             print(f"Bbox: {det[:4]}")
#             print(f"class_id: {det[8]}")
#             print(f"score: {det[4]}")
#             x1, y1, x2, y2 = det[:4]
#             class_id = int(det[8])
#             # score = det[4] * det[5 + class_id]
#             score = det[4]
#             label = f"{CLASS_NAMES[class_id]}: {score:.2f}"

#             plt.gca().add_patch(
#                 plt.Rectangle(
#                     (x1, y1),
#                     x2 - x1,
#                     y2 - y1,
#                     fill=False,
#                     edgecolor=COLORS[class_id],
#                     linewidth=2,
#                 )
#             )
#             plt.text(
#                 x1,
#                 y1 - 5,
#                 label,
#                 fontsize=12,
#                 bbox=dict(facecolor=COLORS[class_id], alpha=1),
#             )

# plt.axis("off")
# plt.show()

tensor_input shape: (1, 3, 640, 640)
predictions shape: (1, 8400, 8)
[array([[[ 7.05275631e+00,  1.88555336e+00,  3.50317535e+01, ...,
          5.27590513e-04,  2.11997658e-01,  1.19525701e-01],
        [ 1.36574211e+01, -3.73984098e-01,  1.90118484e+01, ...,
          6.94990158e-05,  1.09940767e-01,  2.51293182e-02],
        [ 1.60149727e+01, -1.01238787e+00,  4.21233654e+00, ...,
          7.23004341e-05,  2.43362993e-01,  8.53842497e-03],
        ...,
        [ 5.33503296e+02,  5.66761230e+02,  1.69378586e+02, ...,
          1.03354454e-04,  5.80913126e-02,  9.97965634e-02],
        [ 5.58621582e+02,  5.63024353e+02,  9.67023849e+01, ...,
          1.22010708e-04,  3.69730294e-02,  4.18034494e-02],
        [ 5.56138428e+02,  5.63692444e+02,  4.34511337e+01, ...,
          3.03983688e-05,  3.35299969e-02,  6.32097423e-02]]],
      dtype=float32)]
